In [ ]:
import sys
if '/Users/nehiljain/code/find-your-mate-ai/src' not in sys.path:
    sys.path.append('/Users/nehiljain/code/find-your-mate-ai/src')

import nest_asyncio

nest_asyncio.apply()
from find_your_mate_ai.config import settings
from find_your_mate_ai.data_ingestion import *
import pandas as pd
# documents = ingest_profiles_data(
#         source_data_path=settings.SOURCE_DATA_PATH,
#         output_data_path=settings.OUTPUT_DATA_PATH,
#     )


In [ ]:
print(settings.MONGO_URI)
storage_context = StorageContext.from_defaults(
        docstore=MongoDocumentStore.from_uri(uri=settings.MONGO_URI),
    )
docs = storage_context.docstore.docs




In [ ]:
from typing import Dict, List
print("Number of documents: ", len(docs))
nodes: List[BaseNode] = [node for _, node in docs.items()]




In [ ]:
from pymongo import MongoClient
from pymongo.server_api import ServerApi

# Connect to MongoDB
client = MongoClient(settings.MONGO_URI, server_api=ServerApi('1'))
db = client['db_docstore']
collection = db['docstore/data']

def update_document_ids_with_hashes(metadata_df):
    metadata_df = fetch_all_documents_from_mongodb(settings.MONGO_URI, "db_docstore", "docstore/data")
    # Create a clone of the metadata DataFrame
    cloned_df = metadata_df.copy()

    # Modify the _id column to be a hash of the file_name column
    cloned_df['_id'] = cloned_df['file_name'].apply(lambda x: hash(x))

    # Iterate over each row in the DataFrame and update the MongoDB document
    for index, row in cloned_df.iterrows():
        original_id = metadata_df.at[index, '_id']
        new_id = row['_id']

        # Update the document in MongoDB
        update_result = collection.update_one(
            {'_id': original_id},
            {'$set': {'_id': new_id}}
        )

        # Log the result of the update
        if update_result.modified_count == 1:
            logging.info(f"Document with original ID {original_id} updated to new ID {new_id}")
        else:
            logging.error(f"Failed to update document with original ID {original_id}")

# Call the function with the metadata DataFrame
update_document_ids_with_hashes(metadata_df)


# Close the connection
client.close()


In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-program-openai
%pip install llama-index-readers-web


In [ ]:
import nest_asyncio

nest_asyncio.apply()

import os
import openai
from pydantic import BaseModel, Field
from typing import List
class NodeMetadata(BaseModel):
    """Node metadata."""

    entities: List[str] = Field(
        ..., description="Unique entities in this text chunk."
    )
    summary: str = Field(
        ..., description="A concise summary of this text chunk."
    )
    contains_number: bool = Field(
        ...,
        description=(
            "Whether the text chunk contains any numbers (ints, floats, etc.)"
        ),
    )

from llama_index.program.openai import OpenAIPydanticProgram
from llama_index.core.extractors import PydanticProgramExtractor

EXTRACT_TEMPLATE_STR = """\
Here is the content of the section:
----------------
{context_str}
----------------
Given the contextual information, extract out a {class_name} object.\
"""

openai_program = OpenAIPydanticProgram.from_defaults(
    output_cls=NodeMetadata,
    llm=OpenAI(
        temperature=0,
        model="llama3",
        api_base="http://localhost:11434/v1",
        api_key="ollama",
    ),
    prompt_template_str="{input}",
    # extract_template_str=EXTRACT_TEMPLATE_STR
)

program_extractor = PydanticProgramExtractor(
    program=openai_program, input_key="input", show_progress=True
)

In [ ]:
# load in blog

from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.node_parser import SentenceSplitter

reader = SimpleWebPageReader(html_to_text=True)
docs = reader.load_data(urls=["https://eugeneyan.com/writing/llm-patterns/"])

In [ ]:
from llama_index.core.ingestion import IngestionPipeline

node_parser = SentenceSplitter(chunk_size=1024)

pipeline = IngestionPipeline(transformations=[node_parser, program_extractor])

orig_nodes = pipeline.run(documents=docs)

In [ ]:
orig_nodes

In [ ]:
\